<a href="https://colab.research.google.com/github/Orenjonas/natural_language_processing_with_disaster_tweets/blob/main/NLP_disaster_tweets_using_torch_RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
Using a RoBERTa model pretrained on twitter data for classifying tweets as relating to real disasters or not.

Model description can be found [here](https://huggingface.co/cardiffnlp/twitter-roberta-base)

TODO:
- More detailed text cleaning.

# Import data
## Mount google drive to notebook

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

# Import data

In [ ]:
import os
for dirname, _, filenames in os.walk('/gdrive/My Drive/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install datasets
from datasets import load_dataset

raw_dataset = load_dataset('csv', data_files='/gdrive/My Drive/input/nlp-getting-started/train.csv')
raw_competition_test_dataset = load_dataset('csv', data_files='/gdrive/My Drive/input/nlp-getting-started/test.csv')


In [ ]:
raw_dataset

# Preprocess data

In [ ]:
def preprocess(row):
    text = row['text']
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    row['text'] = " ".join(new_text)
    return row
    # return " ".join(new_text)

In [ ]:
# Data dict currently only concist of a 'train' entry.
#  We will later split this into a training and evaluation set
processed_data = raw_dataset['train'].map(preprocess)

## TODO: Further data cleaning

# Tokenize data

In [ ]:
!pip install transformers

from transformers import pipeline, AutoTokenizer

MODEL = "cardiffnlp/twitter-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenized_data = processed_data.map(tokenize_function, batched=True)

# # TODO: adapt code
# MAX_LEN = 
# tokenized_train_data = processed_train_data.map(tokenize_function, batched=True)
# tokenized_test_data = processed_test_data.map(tokenize_function, batched=True)



In [ ]:
# Split training data into a dictionary containing a test and training (validation) set
tokenized_datasets = tokenized_data.train_test_split(test_size=0.2)

In [ ]:
small_train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(1000))
small_eval_dataset =  tokenized_datasets['test'].shuffle(seed=42).select(range(1000))

full_train_dataset =  tokenized_datasets["train"]
full_eval_dataset =   tokenized_datasets["test"]

In [ ]:

encoded_input = tokenizer(text, return_tensors='tf')

# Model

In [ ]:
import tensorflow as tf

train_tf_dataset = small_train_dataset.remove_columns(["text"]).with_format("tensorflow")
eval_tf_dataset  = small_eval_dataset.remove_columns(["text"]).with_format("tensorflow")

def sliceDataset_to_batchDataset(dataset):
    features = {x: dataset[x].to_tensor() for x in tokenizer.model_input_names}
    dataset = tf.data.Dataset.from_tensor_slices((train_features, dataset["target"]))
    return dataset.shuffle(len(dataset)).batch(8)


train_tf_dataset = sliceDataset_to_batchDataset(train_tf_dataset)
eval_tf_dataset = sliceDataset_to_batchDataset(eval_tf_dataset)

In [ ]:
from transformers import TFAutoModelForSequenceClassification

# Initiate a tensorflow model from the pretrained model. Will throw a warning about some layers not bein initialized
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2)

In [ ]:
tf.get_logger().setLevel('ERROR')

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.summary()

In [ ]:

history = model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=3)

## Model is overfitting
- Try more detailed cleaning.
- Try another pretrained model, e.g roberta-base.